In [1]:
#import libraries

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import plot_model
from keras.datasets import mnist

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from PIL import Image


In [2]:
#helper functions

def SE(pred1, pred2): #arguments = 2 arrays (sum of squares)
    return ((pred1-pred2)**2).sum()

def MSE(pred1, pred2): #arguments = 2 arrays (mean squard error)
    return ((pred1-pred2)**2).mean()
    
def Gaussian_Im():
        return (np.random.normal(loc=1, scale=0.5, size=784)*127.5).astype(int)


In [3]:
#importing data
(all_X, all_y), (testall_X, testall_y) = mnist.load_data()

#preprocess data

#discard all images not depicting a 0 or 1
train_X1 = []
train_y1 = []
train_X0 = []
train_y0 = []
for xi in range(len(all_X)):
    if(all_y[xi] == 1):
        train_X1.append(np.reshape(all_X[xi],784))
        train_y1.append(1)
    if(all_y[xi] == 0):
        train_X0.append(np.reshape(all_X[xi],784))
        train_y0.append(0)

test_X1 = []
test_y1 = []
test_X0 = []
test_y0 = []
for xi in range(len(testall_X)):
    if(testall_y[xi] == 1):
        test_X1.append(np.reshape(testall_X[xi],784))
        test_y1.append(1)
    if(testall_y[xi] == 0):
        test_X0.append(np.reshape(testall_X[xi],784))
        test_y0.append(0)
        
train_X0 = np.asarray(train_X0) #train images depicting a 0, correct dimension (784)
train_y0 = np.asarray(train_y0)
train_X1 = np.asarray(train_X1)
train_y1 = np.asarray(train_y1)

test_X0 = np.asarray(test_X0)
test_y0 = np.asarray(test_y0)
test_X1 = np.asarray(test_X1)
test_y1 = np.asarray(test_y1)

#normalize data in [0,1]


In [4]:
#building networks

#target network
target = Sequential()
target.add(keras.Input(shape=(784,)))
target.add(Dense(64, activation='sigmoid', kernel_initializer=keras.initializers.RandomNormal(stddev=2), bias_initializer='random_normal'))
target.add(Dropout(0.2))
target.add(Dense(64, activation='sigmoid', kernel_initializer=keras.initializers.RandomNormal(stddev=2), bias_initializer='random_normal'))
target.add(Dropout(0.2))
target.add(Dense(24,activation='linear', kernel_initializer=keras.initializers.RandomNormal(stddev=2), bias_initializer='random_normal'))

#prediction network (same architecture, different intialization)
prediction = Sequential()
prediction.add(keras.Input(shape=(784,)))
prediction.add(Dense(64, activation='sigmoid', kernel_initializer=keras.initializers.RandomNormal(stddev=2), bias_initializer='zeros'))
target.add(Dropout(0.2))
prediction.add(Dense(64, activation='sigmoid', kernel_initializer=keras.initializers.RandomNormal(stddev=2), bias_initializer='zeros'))
target.add(Dropout(0.2))
prediction.add(Dense(24,activation='linear', kernel_initializer=keras.initializers.RandomNormal(stddev=2), bias_initializer='zeros'))

#prediction.summary()
#plot_model(prediction)

In [5]:
#compute Gaussian Images
gauss = []
for i in range(1000):
    gauss.append(Gaussian_Im())
gauss = np.asarray(gauss)
#print(gauss[0])

#concatenate 0-imgages and gauss-images
train_im = np.concatenate((train_X0[0:1000], gauss))
#train_im = np.concatenate((train_X0[0:1000], gauss, train_X1[0:1000]))
print(train_im.shape)

#compute target values
val_target = target.predict(train_im)
print(val_target.shape)

#pred = target.predict(train_X0[0:2]) * 10
#print("input image: ", train_X0[0].shape)
#print(pred.shape)
#pred

(3000, 784)
(3000, 24)


In [6]:
#predict error of 0-image before training
sum0 = 0
pred1 = prediction.predict(train_X0[1000:1100])
pred2 = target.predict(train_X0[1000:1100])
for i in range(100):
    sum0 = sum0 + MSE(pred1[i], pred2[i])

print('error of 0-image before training:     ', sum0/100)


#predict error of Gauss-image before training
sumG = 0
for i in range(100):
    image = np.asarray([Gaussian_Im()])
    sumG = sumG + MSE(target.predict(image), prediction.predict(image))

print('error of Gauss-image before training: ', sumG/100)


#predict error of 1-image before training
sum1 = 0
pred1 = prediction.predict(train_X1[0:100])
pred2 = target.predict(train_X1[0:100])
for i in range(100):
    sum1 = sum1 + MSE(pred1[i], pred2[i])

print('error of 1-image before training:     ', sum1/100)

error of 0-image before training:      147.10725608825683
error of Gauss-image before training:  185.91091323852538
error of 1-image before training:      204.57093322753906


In [7]:
#training

prediction.compile(
    loss='MSE',
    optimizer='adam'
    #metrics=,
)

prediction.fit(
    train_im,
    val_target,
    epochs=10,
    batch_size=8,
    verbose=2,
    shuffle=True
)

Epoch 1/10
375/375 - 1s - loss: 86.5390
Epoch 2/10
375/375 - 1s - loss: 39.9938
Epoch 3/10
375/375 - 1s - loss: 32.9924
Epoch 4/10
375/375 - 1s - loss: 29.5210
Epoch 5/10
375/375 - 1s - loss: 27.5285
Epoch 6/10
375/375 - 1s - loss: 26.4406
Epoch 7/10
375/375 - 1s - loss: 25.7758
Epoch 8/10
375/375 - 1s - loss: 25.2270
Epoch 9/10
375/375 - 1s - loss: 24.7277
Epoch 10/10
375/375 - 1s - loss: 24.3182


In [8]:
#predict error of 0-image after training
sum0 = 0
pred1 = prediction.predict(train_X0[1000:1100])
pred2 = target.predict(train_X0[1000:1100])
for i in range(100):
    sum0 = sum0 + MSE(pred1[i], pred2[i])

print('error of 0-image before training:     ', sum0/100)


#predict error of Gauss-image after training
sumG = 0
for i in range(100):
    image = np.asarray([Gaussian_Im()])
    sumG = sumG + MSE(target.predict(image), prediction.predict(image))

print('error of Gauss-image before training: ', sumG/100)


#predict error of 1-image after training
sum1 = 0
pred1 = prediction.predict(train_X1[0:100])
pred2 = target.predict(train_X1[0:100])
for i in range(100):
    sum1 = sum1 + MSE(pred1[i], pred2[i])

print('error of 1-image before training:     ', sum1/100)

error of 0-image before training:      28.07582675933838
error of Gauss-image before training:  19.195468554496767
error of 1-image before training:      24.67736753463745
